In [1]:
import argparse
import os
import sys
import tabulate
import time
import torch
import torch.nn.functional as F

import curves
import data
import models
import utils

import numpy as np

import copy

In [2]:
torch.backends.cudnn.benchmark = True

loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    128,
    1,
    "VGG",
    False)

Files already downloaded and verified
Using train (45000) + validation (5000)
Files already downloaded and verified


In [3]:
architecture = getattr(models, "VGG16")
model1 = architecture.base(num_classes=10, **architecture.kwargs)
model2 = architecture.base(num_classes=10, **architecture.kwargs)
model3 = architecture.base(num_classes=10, **architecture.kwargs)
E =  architecture.base(num_classes=10, **architecture.kwargs)
L =  architecture.base(num_classes=10, **architecture.kwargs)

model1.load_state_dict(torch.load("curves/curve50/checkpoint-100.pt")['model_state'])
model2.load_state_dict(torch.load("curves/curve51/checkpoint-100.pt")['model_state'])

In [4]:
for parameter, p1, p2 in zip(L.parameters(), model1.parameters(), model2.parameters()):
    parameter.data.copy_((p1+p2)/2) 

In [5]:
number_points = 50

models = [ architecture.base(num_classes=10, **architecture.kwargs) for i in range(number_points)]

ind = 50
T = True
l = []

for m in models:

    ckpt = 'curves/curve' + str(ind) + '/checkpoint-100.pt'
    checkpoint = torch.load(ckpt)
    m.load_state_dict(checkpoint['model_state'])
    ind+=1
    l.append(ind)

for i, m in enumerate(models):
    for parameter, p in zip(E.parameters(), m.parameters()):
        if i == 0:
            parameter.data.copy_((p))
        else:
            parameter.data.copy_((parameter+p))

for parameter in E.parameters():
    parameter.data.copy_(parameter/len(models))


In [6]:
# list(model3.parameters())[-1]

In [7]:
# list(model1.parameters())[-1]

In [8]:
# list(model2.parameters())[-1]

In [9]:
# list(E.parameters())[-1]

In [10]:
for mu in list(E.parameters())[:]:
    mean = mu.mean(0)
#     print(mean.shape)
    for p in mu:
        p.data.copy_(mean)
#         p = mean

In [11]:
list(E.parameters())[-1].shape, list(E.parameters())[0].shape

(torch.Size([10]), torch.Size([64, 3, 3, 3]))

In [12]:
list(model3.parameters())[-2].shape,

(torch.Size([10, 512]),)

In [13]:
for parameter, p1, p2, mu in zip(model3.parameters(), model1.parameters(), model2.parameters(), E.parameters()):
    parameter.data.copy_((mu+(1/np.sqrt(2))*(p1-mu)+(1/np.sqrt(2))*(p2-mu))) 

In [14]:
criterion = F.cross_entropy
regularizer = None 
optimizer = torch.optim.SGD(
    filter(lambda param: param.requires_grad, model3.parameters()),
    lr=0.01,
    momentum=0.9,
    weight_decay=0.0
)

In [15]:
def dist(model1, model2):
    par1 = np.concatenate([p.data.cpu().numpy().ravel() for p in model1.parameters()])
    par2 = np.concatenate([p.data.cpu().numpy().ravel() for p in model2.parameters()])
    u = par2 - par1
    dx = np.linalg.norm(u)
#     print('dimention', par1.shape)
    print('distance: %5.4f' % dx)
    return dx

In [16]:
print('distance between middle point and end points')
dist(model1, model2)
dist(model1, L)
dist(model2, L)
print('distance between E point and end points')
dist(model1, E)
dist(model2, E)
print('distance between model point and end points')
dist(model1, model3)
dist(model2, model3)
print('distance between middle point and model')
dist(model3, L)

distance between middle point and end points
distance: 112.7420
distance: 56.3710
distance: 56.3710
distance between E point and end points
distance: 79.7682
distance: 79.7654
distance between model point and end points
distance: 61.0251
distance: 61.0267
distance between middle point and model
distance: 23.3767


23.376654

In [17]:
model3.cuda();
train_res = utils.test(loaders['train'], model3, criterion, regularizer)
test_res = utils.test(loaders['test'], model3, criterion, regularizer)

columns = ['tr_loss', 'tr_acc', 'te_nll', 'te_acc']

values = [train_res['loss'], train_res['accuracy'], test_res['nll'],
          test_res['accuracy']]

table = tabulate.tabulate([values], columns, tablefmt='simple', floatfmt='9.4f')

print(table)

  tr_loss     tr_acc     te_nll     te_acc
---------  ---------  ---------  ---------
   3.7481    10.2511     3.7607    10.7800


In [ ]:
import argparse
import os
import sys
import tabulate
import time
import torch
import torch.nn.functional as F

import curves
import data
import models
import utils
import copy

In [ ]:
checkpoints = torch.load("curves/curve50/checkpoint-100.pt")

In [18]:
utils.save_checkpoint(
            'curves/curve50',
            100,
            name='E',
            model_state=E.state_dict(),
            optimizer_state=checkpoints['optimizer_state']
        )

In [19]:
t = torch.tensor([3, 6,7])

In [22]:
t.cpu().numpy()


array([3, 6, 7])

In [13]:
import numpy as np

In [24]:
t = 6
if isinstance(t, (int, float)):
              print('fdcfc')

fdcfc


In [16]:
t = list(6) #np.array([6,7])
if isinstance(t, ):
    print('ddc')

TypeError: 'int' object is not iterable

In [11]:
for i in t:
    print(t)

TypeError: 'int' object is not iterable